<a href="https://colab.research.google.com/github/YuehMintTai/000/blob/main/MongoDBTest20220201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,make_response
app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
  return '哈囉你好嗎?'

if __name__=='__main__':
  app.run()